In [5]:
pip install statsmodels

  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates
import statsmodels.api as sm

In [3]:
df = pd.read_csv('health.csv')
df.columns = ['date','click']
df

,date,click
0,2019-01-01,36
1,2019-01-02,45
2,2019-01-03,41
3,2019-01-04,40
4,2019-01-05,40
...,...,...
360,2019-12-27,86
361,2019-12-28,80
362,2019-12-29,87
363,2019-12-30,88


### 시계열 데이터의 조건
- 1년 이상 수집(계절성 반영)
- 일정한 간격으로 빠짐없이 데이터가 수집되어야 함
- 선형성 전제 [선형성, 독립성, 등분산성, 정규성]

In [4]:
df['click'].isnull().sum()

0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   click   365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [8]:
# date가 시계열인지 확인
type(df['date'][0])

str

In [9]:
df['date'] = pd.to_datetime(df['date'])
type(df['date'][0])

pandas._libs.tslibs.timestamps.Timestamp